# 2. Mapping: Mean-State and Seasonal Difference Plots
### Authors

Samantha Stevenson sstevenson@ucsb.edu

### Table of Contents

[Goals](#purpose)

[Import Packages, Define Time Periods and Regions](#setup)

[Read in Data and Concatenate Data Arrays](#data_io)

[Calculate Time Averages Over Specified Periods, Seasons](#time_avg)

[Plot the Mean State: Specified Region](#map)


<a id='purpose'></a> 
## **Goals**

In this tutorial, we will be creating difference maps to show how patterns of surface air temperature have changed over time. This tutorial is an extension of tutorial 1 in this repo. New skills encountered in this tutorial:

1. _Concatenating_ data from multiple netCDF files

2. Plotting data from different _seasons_


We'll be working with data from MULTIPLE files today! Direct links are below:

#### [File 1 for Today: Historical tas from CanESM5 member r10i1p1f1](https://drive.google.com/file/d/1TaWe1hs6Lgr-AEM1y74xGA3EBaECyMMV/view?usp=sharing)

#### [File 2 for Today: SSP370 tas from CanESM5 member r10i1p1f1](https://drive.google.com/file/d/1WDAWoepRTliqkFmQMF9D8jF0e_FwA2KG/view?usp=drive_link)

These files can also be downloaded from the [Earth System Grid Federation MetaGrid site](https://aims2.llnl.gov) (see the [Web download tutorial](https://climate-datalab.org/data-download-from-web/) if you need more information on how to do this).

<a id='setup'></a> 
## **Import Packages, Define Time Periods and Regions**

We'll import the necessary packages first, which are the same as the ones we used in tutorial 1:

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature

Before we go to the step of reading in the data, we'll *first* set up some _parameters_ that the rest of the notebook will use. It's good to set these up front, where they're hard to miss! We'll specify two arrays, which hold:

- our _region of interest_ for plotting: this is specified in the format [minimum latitude, maximum latitude, minimum longitude, maximum longitude] but you can change this around if you want
- our _time periods of interest_ which will be used later for averaging: this is specified in the format [start year, end year] with one array for each time period

**NOTE: specifying these as arrays is not required!** If you're more comfortable with "hard coding" the numbers in, that's fine - we're just including this here to get you used to some coding best practices.

In the example below, the region of interest is set to cover the continental United States, and two time periods are defined:
- Early (`per_early`): the last 50 years up till now
- Late (`per_late`): the next 50 years

In [4]:
# Array with region of interest
# [lat_min, lat_max, lon_min, lon_max]
region = [25, 51, 230, 295]

# Array with time periods of interest
# [starting year, ending year]
per_early = [1975, 2025]
per_late = [2026, 2075]

<a id='data_io'></a> 
## **Read in Data and Concatenate Data Arrays**

We next need to read in our data. But now we'll do it slightly differently than in tutorial 1, since we need to read in data from multiple files.

#### Why are multiple files necessary?

Take a look at the filenames for the two files provided here:
- tas_Amon_CanESM5_historical_r10i1p1f1_gn_185001-201412.nc
- tas_Amon_CanESM5_ssp370_r10i1p1f1_gn_201501-210012.nc

The date strings at the end of the filenames tell you what years are included in each (for more information on the filenames, also refer to our [filename decoder](http://www.climate-datalab.org/filename-decoder)). The historical data file contains years 1850-2014, and the future projection (here, SSP3-7.0) contains years 2015-2100. Since we're trying to average data over the period 1975-2025, that means that we need information from both data files! And of course, the future part of the record is contained entirely in the second data file.

#### Concatenation: xarray `concat`
The easiest way to deal with this is to read in BOTH data files as xarray Datasets, then concatenate them. xarray has a built-in function to handle this, called [concat](https://docs.xarray.dev/en/stable/generated/xarray.concat.html):


In [6]:
# Read in data as xarray Dataset
# historical
data_hist = xr.open_dataset('/Users/samstevenson/Box Sync/Climate_DataLab/tas_Amon_CanESM5_historical_r10i1p1f1_gn_185001-201412.nc')

#ssp370
data_ssp = xr.open_dataset('/Users/samstevenson/Box Sync/Climate_DataLab/tas_Amon_CanESM5_ssp370_r10i1p1f1_gn_201501-210012.nc')

# concatenate
data = xr.concat([data_hist, data_ssp], dim="time")

To make sure that the concat function did what it's supposed to do, we can do some printing of the various datasets:

In [9]:
# Print data
print("Historical data file:")
print(data_hist)

Historical data file:
<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lat: 64, lon: 128)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/53)
    CCCma_model_hash:            55f484f90aff0e32c5a8e92a42c6b9ae7ffe6224
    CCCma_parent_runid:          rc3.1-pictrl
    CCCma_pycmor_hash:           33c30511acc319a98240633965a04ca99c26427e
    CCCma_runid:                 rc3.1-his10
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   1850:01:01:00
    ...                          ...
    tracking_id:              

From this, we see that the historical data file contains information for 1980 months, which makes sense for a file that's supposed to contain monthly averages over 165 years.

We can do the same thing for the SSP370 data file:

In [10]:
print("SSP370 data file:")
print(data_ssp)

SSP370 data file:
<xarray.Dataset>
Dimensions:    (time: 1032, bnds: 2, lat: 64, lon: 128)
Coordinates:
  * time       (time) object 2015-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/53)
    CCCma_model_hash:            24718c8346665b218729640ffe79d263b76456c4
    CCCma_parent_runid:          rc3.1-his10
    CCCma_pycmor_hash:           33c30511acc319a98240633965a04ca99c26427e
    CCCma_runid:                 rc3.1-s7010
    Conventions:                 CF-1.7 CMIP-6.2
    YMDH_branch_time_in_child:   2015:01:01:00
    ...                          ...
    tracking_id:                 hd

For comparison, the concatenated data file has a longer time dimension, that's equal to the sum of the lengths of the `time` coordinates in the historical and SSP370 files:

In [11]:
print("Concatenated data file:")
print(data)

Concatenated data file:
<xarray.Dataset>
Dimensions:    (time: 3012, bnds: 2, lat: 64, lon: 128)
Coordinates:
  * time       (time) object 1850-01-16 12:00:00 ... 2100-12-16 12:00:00
  * lat        (lat) float64 -87.86 -85.1 -82.31 -79.53 ... 82.31 85.1 87.86
  * lon        (lon) float64 0.0 2.812 5.625 8.438 ... 348.8 351.6 354.4 357.2
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object 1850-01-01 00:00:00 ... 2101-01-01 00:00:00
    lat_bnds   (time, lat, bnds) float64 -90.0 -86.58 -86.58 ... 86.58 90.0
    lon_bnds   (time, lon, bnds) float64 -1.406 1.406 1.406 ... 355.8 358.6
    tas        (time, lat, lon) float32 250.9 250.7 250.4 ... 275.7 275.7 275.7
Attributes: (12/53)
    CCCma_model_hash:            55f484f90aff0e32c5a8e92a42c6b9ae7ffe6224
    CCCma_parent_runid:          rc3.1-pictrl
    CCCma_pycmor_hash:           33c30511acc319a98240633965a04ca99c26427e
    CCCma_runid:                 rc3.1-his10
    Convent

<a id='time_avg'></a> 
## **Calculate Time Averages Over Specified Periods, Seasons**

Now we can proceed with the rest of our analysis, following the same steps as before!

We use the `.sel` and `slice` commands to create subsetted data over the two time periods we're interested in. 

**Note: here we specify the inputs to slice using the `per_early` and `per_late` arrays we defined at the beginning!**

Then we just average over the time dimension to get our two-dimensional datasets to use in the plot!

In [12]:
# Select data within the ranges of years specified above

# Early period
data_early = data.sel(time=slice(str(per_early[0])+"-01-01", str(per_early[1])+"-12-31"))

# Late period
data_late = data.sel(time=slice(str(per_late[0])+"-01-01", str(per_late[1])+"-12-31"))

In [15]:
# Calculate time averages
data_early_mn = data_early.mean(dim="time")
data_late_mn = data_late.mean(dim="time")